## Import Libraries

In [1]:
# Standard libraries
import pandas as pd
import numpy as np

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# xgboost
from xgboost import XGBClassifier

# talib
from talib import BBANDS
from talib import SAR
from talib import RSI
from talib import STOCH
from talib import EMA
from talib import WILLR

/opt/anaconda3/envs/python3_8/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## Data Import

In [2]:
train_df = pd.read_csv("training.csv", names=("open", "high", "low", "close"))
test_df = pd.read_csv("testing.csv", names=("open", "high", "low", "close"))
len(test_df)

20

In [3]:
n_days_in = 20
n_days_out = len(test_df)

## Data Preprocessing

In [4]:
# MinMax Normalization
maxValue = train_df.to_numpy().max()
minValue = train_df.to_numpy().min()
diff = maxValue - minValue
train = train_df.transform(lambda x: (x - minValue) / diff)
test = test_df.transform(lambda x: (x - minValue) / diff)
train

,open,high,low,close
0,0.705354,0.725354,0.697980,0.701010
1,0.693636,0.701313,0.687273,0.693333
2,0.685960,0.688182,0.668687,0.684444
3,0.681717,0.682626,0.660707,0.663030
4,0.659596,0.660303,0.640101,0.653434
...,...,...,...,...
1483,0.354040,0.356566,0.347778,0.352929
1484,0.355152,0.359495,0.350707,0.354343
1485,0.358081,0.363939,0.351515,0.359293
1486,0.362727,0.370808,0.356263,0.370202


In [5]:
train["upperband"], train["middleband"], train["lowerband"] = BBANDS(train.close.to_numpy())
train["sar"] = SAR(train.high.to_numpy(), train.low.to_numpy())
train["rsi"] = RSI(train.close.to_numpy(), timeperiod=5)
train["slowk"], train["slowd"] = STOCH(train.high.to_numpy(), train.low.to_numpy(), train.close.to_numpy())
train["ema"] = EMA(train.close.to_numpy(), timeperiod=5)
train["willr"] = WILLR(train.high.to_numpy(), train.low.to_numpy(), train.close.to_numpy(), timeperiod=9)
train.isnull().sum()

open          0
high          0
low           0
close         0
upperband     4
middleband    4
lowerband     4
sar           1
rsi           5
slowk         8
slowd         8
ema           4
willr         8
dtype: int64

In [6]:
train_data = train.dropna()
train_data = train_data.reset_index(drop=True)
train_data

,open,high,low,close,upperband,middleband,lowerband,sar,rsi,slowk,slowd,ema,willr
0,0.632121,0.633434,0.610606,0.628889,0.669042,0.647333,0.625624,0.693245,10.760278,35.685147,30.969656,0.648791,-84.066901
1,0.641010,0.657576,0.630505,0.637374,0.666021,0.644121,0.622222,0.683329,24.037590,35.196490,34.271958,0.644985,-70.489978
2,0.635657,0.653535,0.632323,0.648182,0.665042,0.643636,0.622231,0.674602,38.586495,46.049189,38.976942,0.646051,-51.562500
3,0.655455,0.661212,0.640909,0.642626,0.653047,0.640000,0.626953,0.666922,34.358325,57.668859,46.304846,0.644909,-55.539972
4,0.695657,0.727980,0.685354,0.723434,0.724624,0.656101,0.587578,0.610606,78.063128,75.844229,59.854092,0.671084,-3.872633
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,0.354040,0.356566,0.347778,0.352929,0.367668,0.344525,0.321382,0.302071,67.065605,87.132992,83.501390,0.345742,-9.775641
1476,0.355152,0.359495,0.350707,0.354343,0.361821,0.350606,0.339391,0.304352,68.454615,83.240226,84.920261,0.348609,-8.121019
1477,0.358081,0.363939,0.351515,0.359293,0.359533,0.354566,0.349598,0.307660,73.368544,78.956152,83.109790,0.352170,-6.845238
1478,0.362727,0.370808,0.356263,0.370202,0.371149,0.357798,0.344447,0.312163,81.365806,83.695902,81.964093,0.358181,-0.981997


## Training

In [7]:
y = list()
for i in range(len(train_data)):
    isBull = (train_data["open"][i] > train_data["sar"][i], 
              train_data["open"][i] >= train_data["middleband"][i],
              train_data["rsi"][i] > 50,
              train_data["slowk"][i] >= train_data["slowd"][i],
              train_data["open"][i] >= train_data["ema"][i],
              train_data["willr"][i] > -50)
    if np.count_nonzero(isBull) > 4:
        y.append(2)
    elif np.count_nonzero(isBull) < 2:
        y.append(0)
    else:
        y.append(1)
y = np.array(y, dtype=int)

In [8]:
X = list()
for i in range(n_days_in, len(train_data)):
    X.append(train_data.loc[i-n_days_in:i-1, :].values)
X = np.array(X)
X

array([[[  0.63212121,   0.63343434,   0.61060606, ...,  30.96965635,
           0.64879112, -84.06690141],
        [  0.6410101 ,   0.65757576,   0.63050505, ...,  34.27195808,
           0.64498533, -70.48997773],
        [  0.63565657,   0.65353535,   0.63232323, ...,  38.97694176,
           0.64605082, -51.5625    ],
        ...,
        [  0.76646465,   0.77454545,   0.75585859, ...,  59.08550642,
           0.76710405, -28.2245827 ],
        [  0.7689899 ,   0.78343434,   0.76414141, ...,  57.61181757,
           0.76806937, -25.49317147],
        [  0.75616162,   0.76292929,   0.75131313, ...,  52.80309764,
           0.76632234, -62.72965879]],

       [[  0.6410101 ,   0.65757576,   0.63050505, ...,  34.27195808,
           0.64498533, -70.48997773],
        [  0.63565657,   0.65353535,   0.63232323, ...,  38.97694176,
           0.64605082, -51.5625    ],
        [  0.65545455,   0.66121212,   0.64090909, ...,  46.30484588,
           0.6449093 , -55.53997195],
        ...,


In [9]:
y = y[n_days_in + n_days_out :]
len(y)

1440

In [10]:
test = X[-len(test_df):]
len(test)

20

In [11]:
new_X = X[:-len(test_df)]

new_X = new_X.reshape((len(y), -1))

In [12]:
new_X.shape

(1440, 260)

In [13]:
X_train, X_val, y_train, y_val = train_test_split(new_X, y, test_size=0.2, shuffle=False)

In [14]:
xgb = XGBClassifier(learning_rate=0.1, 
                    objective='multi:softmax',
                    num_class=3,
                    n_estimators=1000, max_depth=1, min_child_weight=2, use_label_encoder=False)
# model = xgb.fit(X_train, y_train,
#                eval_set=[(X_val, y_val)],
#               eval_metric="auc",
#                verbose=True)

In [15]:
parameters = {
    'max_depth': list(range(1, 10)),
    'min_child_weight': list(range(1, 10)),
    "n_estimators": list(range(100, 1001, 100))
}
# gsearch = GridSearchCV(xgb, param_grid=parameters, scoring="f1", cv=2)
# gsearch.fit(X_train, y_train,  eval_set=[(X_val, y_val)], eval_metric="auc", verbose=True)
# best_parameters = gsearch.best_estimator_.get_params()
rsearch = RandomizedSearchCV(xgb, param_distributions=parameters, scoring="f1", cv=2)
rsearch.fit(X_train, y_train,  eval_set=[(X_val, y_val)], eval_metric="auc", verbose=False)
best_parameters = rsearch.best_estimator_.get_params()

/opt/anaconda3/envs/python3_8/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/envs/python3_8/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/anaconda3/envs/python3_8/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/opt/anaconda3/envs/python3_8/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/opt/anaconda3/envs/python3_8/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 1123, in f1_score
    return fbeta_score(
  File "/opt/anaconda3/envs/python3_8/lib/python3.8/site-packages/sklearn/metrics/_classifi

In [ ]:
print(best_parameters)

{'objective': 'multi:softprob', 'use_label_encoder': False, 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'enable_categorical': False, 'gamma': 0, 'gpu_id': -1, 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 2, 'min_child_weight': 3, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 500, 'n_jobs': 8, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': None, 'subsample': 1, 'tree_method': 'exact', 'validate_parameters': 1, 'verbosity': None, 'num_class': 3}


In [ ]:
xgb = XGBClassifier(learning_rate=0.1, n_estimators=30,
                    objective='multi:softmax',
                    num_class=3,
                    max_depth=3, min_child_weight=10, use_label_encoder=False)
model = xgb.fit(X_train, y_train,
                eval_set=[(X_val, y_val)],
                eval_metric="mlogloss",
                verbose=True)

[0]	validation_0-mlogloss:1.09864
[1]	validation_0-mlogloss:1.10022
[2]	validation_0-mlogloss:1.10243
[3]	validation_0-mlogloss:1.10245
[4]	validation_0-mlogloss:1.10389
[5]	validation_0-mlogloss:1.10473
[6]	validation_0-mlogloss:1.10708
[7]	validation_0-mlogloss:1.10828
[8]	validation_0-mlogloss:1.10810
[9]	validation_0-mlogloss:1.10996
[10]	validation_0-mlogloss:1.10929
[11]	validation_0-mlogloss:1.11130
[12]	validation_0-mlogloss:1.11109
[13]	validation_0-mlogloss:1.11344
[14]	validation_0-mlogloss:1.11421
[15]	validation_0-mlogloss:1.11624
[16]	validation_0-mlogloss:1.11746
[17]	validation_0-mlogloss:1.11796
[18]	validation_0-mlogloss:1.12061
[19]	validation_0-mlogloss:1.12049
[20]	validation_0-mlogloss:1.12248
[21]	validation_0-mlogloss:1.12356
[22]	validation_0-mlogloss:1.12440
[23]	validation_0-mlogloss:1.12404
[24]	validation_0-mlogloss:1.12652
[25]	validation_0-mlogloss:1.12703
[26]	validation_0-mlogloss:1.12935
[27]	validation_0-mlogloss:1.13017
[28]	validation_0-mlogloss:1.1

In [ ]:
model.predict(test.reshape(20, -1))

array([1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 0, 2, 2, 0, 0])

In [ ]:
test["upperband"], test["middleband"], test["lowerband"] = BBANDS(test.close.to_numpy())
# train["ma5"] = MA(train.close.to_numpy(), timeperiod=5)
test["sar"] = SAR(test.high.to_numpy(), test.low.to_numpy())
test["rsi"] = RSI(test.close.to_numpy(), timeperiod=5)
test["slowk"], test["slowd"] = STOCH(test.high.to_numpy(), test.low.to_numpy(), test.close.to_numpy())

AttributeError: 'numpy.ndarray' object has no attribute 'close'

In [ ]:
predictions = model.predict(test.values)
len(predictions)

In [ ]:
ans = []
val = 0
for i in range(1, len(predictions)):
    _sum = sum(predictions[i-1:i+1])
    if _sum == 2:
        val = 1 if (val == 1) else (val + 1) 
    elif _sum == 1:
        val = val
    else:
        val = -1 if (val == -1) else val - 1
    ans.append(val)
print(ans, len(ans), sep='\n\n')
with open("./output.csv", "w") as fp:
    for i in range(len(ans)):
        print(ans[i], file=fp)